In [1]:
import sys
sys.path.append('/public/src')
from factor_evaluation_server import FactorEvaluation,DataService # type: ignore
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import pandas_ta as ta

In [2]:
ds=DataService()
df=ds['ETHUSDT_15m_2020_2025']['2021-10-01':]

In [3]:
# 添加缺失的vwap字段（如果未提供）
if 'vwap' not in df.columns:
    df['vwap'] = df['turnover'] / df['volume']

In [4]:
evaluator=FactorEvaluation(df=df,future_return_periods=10)

# 定义因子！

In [5]:
def calculate_kup1_rsi_composite(df, rsi_window=14, vol_window=5, atr_window=10):
    """
    复合确认的尾部信号因子
    结合成交量、资金流向和波动率多维度验证尾部信号
    """
    # 计算RSI
    df['rsi'] = ta.rsi(df['close'], length=rsi_window)
    overbought = np.where(df['rsi'] > 70, 1.5, 0.5)
    
    # 基础KUP1计算
    max_open_close = np.maximum(df['open'], df['close'])
    upper_shadow = df['high'] - max_open_close
    base_kup1 = upper_shadow / df['open']
    
    # 1. 成交量分析
    vol_ma = df['volume'].rolling(vol_window).mean()
    vol_ratio = df['volume'] / vol_ma
    
    # 2. 资金流向分析
    taker_sell_volume = df['volume'] - df['taker_buy_volume']
    net_flow = (df['taker_buy_volume'] - taker_sell_volume) / df['volume']
    
    # 3. 波动率分析
    df['atr'] = ta.atr(df['high'], df['low'], df['close'], length=atr_window)
    atr_rank = df['atr'].rolling(50).rank(pct=True)
    
    # 复合条件：同时满足三个尾部强化条件
    shadow_rank = base_kup1.rank(pct=True)
    tail_condition = (
        (shadow_rank < 0.2) & 
        (vol_ratio > 1.4) & 
        (net_flow < -0.1) & 
        (atr_rank < 0.4)
    )
    
    # 信号强化
    enhancement = np.where(tail_condition, 2.5, 1.0)
    
    return base_kup1 * overbought * enhancement * np.where(tail_condition, -1, 1)

# 因子测试

In [6]:
evaluator.set_factor(
    factor_func=calculate_kup1_rsi_composite,
    factor_name='calculate_kup1_rsi_composite'
)

In [7]:
result=evaluator.run_full_evaluation(run_stationarity_test=False)


📊 单币种 (single) 详细评估结果:
--------------------------------------------------
🔗 相关性分析:
   IC (Pearson): 0.011957
   Rank_IC (Spearman): 0.010436
📊 信息比率:
   IR: -0.142300
   有效分组数: 10
📊 因子分布:
📋 数据概况:
   数据长度: 130731
   因子列: calculate_kup1_rsi_composite
   收益率列: future_return
   未来收益周期: 10
--------------------------------------------------

🖼️  单币种 (single) 图片展示:
----------------------------------------
📊 显示分组分析图...


📈 显示因子分布图...
✅ 已显示 2 个图片
----------------------------------------
